In [27]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from urllib.request import urlopen
from urllib.parse import urlparse
import json
from bs4 import BeautifulSoup as bs
import os
import datetime
import MeCab
import pickle
from Classifier import naivebayes

In [2]:
def get_driver():
    driver_path = os.getcwd() + "/chromedriver"
    options = webdriver.ChromeOptions()
    options.add_argument("--mute-audio")
    driver = webdriver.Chrome(driver_path,chrome_options=options)
    driver.maximize_window()
    return driver

In [3]:
def get_url_list(driver,url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,"d-table-cell")))
    trs = driver.find_elements_by_class_name("d-md-none")
    url_list = []
    for tr in trs:
            link = tr.find_element_by_tag_name("a")
            article_url = link.get_attribute("href")
            if "https://alis.to/" in article_url:
                url_list.append(article_url)
    return url_list

In [4]:
def get_text_list(url_list):
    api_url = "https://alis.to/api/articles/%s"
    text_list = []
    for url in url_list:
        parsed = urlparse(url)
        article_id = parsed.path.split("/")[3]
        raw_article_data  = urlopen(api_url % article_id).read().decode("utf-8")
        json_article_data = json.loads(raw_article_data)                    
        body     = json_article_data["body"]
        text = bs(body,"html5lib",from_encoding="utf-8").text.replace("\n","").replace(" ", "").replace("\u3000","")
        text_list.append(text)
    return text_list

In [5]:
url = "https://alis.ocrybit.com/topic/?id=tech"
driver = get_driver()
url_list = get_url_list(driver,url)
tech_text_list = get_text_list(url_list)

/Users/gaxi/.pyenv/versions/3.5.2/envs/jupyter/lib/python3.5/site-packages/bs4/__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [11]:
from get_ranking import get_ranking

In [12]:
df = get_ranking(27)

In [21]:
nontech_url_list = []
for i in df.iterrows():
    url = i[1]["url"]
    if url not in url_list:
        nontech_url_list.append(url)

In [24]:
nontech_text_list = get_text_list(nontech_url_list)

/Users/gaxi/.pyenv/versions/3.5.2/envs/jupyter/lib/python3.5/site-packages/bs4/__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [110]:
def train_texts(cl,text_list,cat):
    for text in text_list:
        cl.train(text,cat)

In [118]:
cl = naivebayes()
train_texts(cl,tech_text_list,"tech")
train_texts(cl,nontech_text_list,"non")

In [125]:
with open('category_count.pickle', mode='wb') as f:
    pickle.dump(cl.cc, f)
with open('feature_count.pickle', mode='wb') as f:
    pickle.dump(cl.fc, f)